In [1]:
'''
(C) Renata Wong 2023

Qiskit code for testing fidelity of derandomised classical shadow on the ground state energy of molecules. 
This notebook implements an optimization: since the derandomized Hamiltonian may contan very few terms,
instead of generating a quantum circuit for each and measuring once, we generate a single circuit and specify 
a shot number that matches the number of occurrences of a derandomized operator. This speeds up the computation 
significantly. 

Procedure:
1. Derandomize the molecule-in-question's Hamiltonian.
2. Choose a variational ansatz with initial parameters selected at random.
3. Apply the derandomized Hamiltonian as basis change operators to the ansatz.
4. Measure the ansatz in the Pauli Z basis and store the results as a shadow.
5. Obtain the expectation value of the molecular Hamiltonian from the shadow.
6. Optimize for minimum Hamiltonian expectation value. 
7. Feed the calculated angles/parameters back to the ansatz.
8. Repeat steps 3-7 till the optimization is completed. 
9. Output the minimized expectation value of the molecular Hamiltonian and the mean-square-root-error. 

Note: Below we perform calculations on the molecular Hamiltonian of H_2.
To perform calculations on other molecules, you will need to specify their geometry, charge and spin 
to replace the values in the driver. 

To do: Hamiltonian for H_2 on 8 qubits using 6-31G basis.
'''

import numpy as np
import matplotlib.pyplot as plt
import time

from functools import partial

from qiskit.circuit.library import EfficientSU2

from qiskit_aer import QasmSimulator
from qiskit import QuantumCircuit, execute

from qiskit.algorithms.optimizers import SLSQP, COBYLA, SPSA

from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

from collections import Counter

from qiskit_nature.second_q.mappers import BravyiKitaevMapper, QubitConverter
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit.opflow import I, StateFn, CircuitStateFn

from predicting_quantum_properties.data_acquisition_shadow import derandomized_classical_shadow
from predicting_quantum_properties.prediction_shadow import estimate_exp

from modified_derandomization import modified_derandomized_classical_shadow





# SPECIFY THE NUMBER OF EXPERIMENTS YOU WANT TO RUN
num_experiments = 10

# SPECIFY THE EXPECTED GROUND STATE ENERGY FOR THE MOLECULE OF INTEREST
EXPECTED_EIGENVALUE = -1.86



# SPECIFY THE GEOMETRY OF THE MOLECULE IN QUESTION
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="6-31g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)


problem = driver.run()
hamiltonian = problem.hamiltonian

# The electronic Hamiltonian of the system
second_q_op = hamiltonian.second_q_op()

# Solving the electronic structure problem = determine the ground state energy of the molecule
from qiskit_nature.second_q.algorithms import GroundStateEigensolver, NumPyMinimumEigensolverFactory
from qiskit_nature.second_q.mappers import BravyiKitaevMapper

# The Bravyi-Kitaev repserentation of the Fermionic Hamiltonian
mapper = BravyiKitaevMapper()
bkenc_hamiltonian = mapper.map(second_q_op)

print(bkenc_hamiltonian)

/home/renata/.local/lib/python3.8/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


1.5280210592332653 * IIIIIIII
- 0.27221525735100427 * IIIIIIIZ
+ 0.08812294324841621 * IIIIIYYX
- 0.08812294324841619 * IIIIIXYY
- 0.40516889844632464 * IIIIIIZZ
+ 0.0885220615853346 * IIIIIIZI
- 0.052063680664412304 * IIIIZIXZ
+ 0.052063680664412304 * IIIIIZXI
+ 0.016033305580191143 * IIIIZIXI
- 0.016033305580191143 * IIIIIZXZ
- 0.6893596033215874 * IIIIIZII
+ 0.10571999330831905 * IIIIIZIZ
- 1.0422225890818186 * IIIIZZZI
+ 0.13179934521156547 * IIIIZZZZ
- 0.01735600095509325 * IIIIIYXY
- 0.01735600095509325 * IIIIIXXX
+ 0.012970529275602022 * IIIIZYZY
+ 0.015318666338180964 * IIIIZXIX
- 0.0023481370625789287 * IIIIIXZX
- 0.0023481370625789287 * IIIIZXZX
+ 0.015318666338180964 * IIIIIXIX
+ 0.012970529275602022 * IIIIIYZY
+ 0.019831904813811668 * IIIIZXXX
+ 0.019831904813811668 * IIIIZYXY
- 0.27221525735100405 * IIIZIIII
+ 0.16302461641468577 * IIIZIIIZ
- 0.04197705526657951 * IIIZIYYX
+ 0.04197705526657951 * IIIZIXYY
+ 0.01986207955163348 * IIZXIIZX
- 0.01986207955163348 * IIIXIIZX
- 

/home/renata/.local/lib/python3.8/site-packages/qiskit_nature/deprecation.py:297: PauliSumOpDeprecationWarning: PauliSumOp is deprecated as of version 0.6.0 and support for them will be removed no sooner than 3 months after the release. Instead, use SparsePauliOp. You can switch to SparsePauliOp immediately, by setting `qiskit_nature.settings.use_pauli_sum_op` to `False`.
  return func(*args, **kwargs)


In [2]:
'''
Use classical eigensolver to obtain the ground state energy for the molecule of interest. 
This value is ca. -1.86 and will be used for evaluating the accuracy of the results found by the quantum-classical method.
'''

converter = QubitConverter(BravyiKitaevMapper())
numpy_solver = NumPyMinimumEigensolver()   

calc = GroundStateEigensolver(converter, numpy_solver)
res = calc.solve(problem)
print('Electronic ground state energy found using classical eigensolver:\n', res) 

/tmp/ipykernel_279/690678606.py:6: DeprecationWarning: The QubitConverter class is deprecated as of version 0.6.0 and will be removed no sooner than 3 months after the release . Instead you should directly use the QubitMapper instance which you used to pass into the QubitConverter as the first argument. Refer to the documentation of the qiskit_nature.second_q.mappers module for more information.
  converter = QubitConverter(BravyiKitaevMapper())


Electronic ground state energy found using classical eigensolver:
 === GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.871583314386
  - computed part:      -1.871583314386
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.151614319937
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.388948701555]
    - computed part:      [0.0  0.0  1.388948701555]
  > Dipole moment (a.u.): [0.0  0.0  -0.000000001555]  Total: 0.000000001555
                 (debye): [0.0  0.0  -0.000000003953]  Total: 0.000000003953
 


In [4]:
'''
Format Hamiltonian terms and coefficients as required by the package predicting-quantum-properties. 
This includes removing all Pauli-I operators.
'''

hamiltonian_terms = []
weights = []

for observable in bkenc_hamiltonian:
    
    observable_str = str(observable)
    observable_str_clean = observable_str.strip()  # removes white spaces
    pauli_str_list = observable_str_clean.split('*')
    tuple_list = []
    
    for op_index, pauli_op in enumerate(pauli_str_list[1]):
        if pauli_op == 'I' or pauli_op == 'X' or pauli_op == 'Y' or pauli_op == 'Z':
            tuple_list.append((pauli_op, op_index-1))
    if len(tuple_list) > 0:
        hamiltonian_terms.append(tuple_list)
        weights.append(float(pauli_str_list[0].strip()))

system_size = len(hamiltonian_terms[0])





hamiltonian_terms_XYZE = []

for term in hamiltonian_terms:
    term_XYZE = []
    for pauli in term:
        if pauli[0] != 'I':
            term_XYZE.append(pauli)
    hamiltonian_terms_XYZE.append(term_XYZE)   
    

weights_XYZ = weights.copy()
weights_XYZ.pop(0)

abs_weights = [abs(weight) for weight in weights]

hamiltonian_terms_XYZ = []
for idx, term in enumerate(hamiltonian_terms_XYZE):
    if term:
        hamiltonian_terms_XYZ.append(term)
    else:
        abs_weights.pop(idx)

        
        
reps = 1   
ansatz = EfficientSU2(system_size, su2_gates=['rx', 'ry'], entanglement='circular', reps=reps, skip_final_rotation_layer=False)  
    

In [5]:
'''
Define the cost function = the expectation value of the Hamiltonian H. 
Since the Hamiltonian is processed term by term, the expectation value is composed as follows:
expval(H) = sum_i (weight_i * expval(term_i))
'''


backend = QasmSimulator(method='statevector', shots=1)


# generate a circuit with just a single layer of randomised basis change operators
# this circuit is appended to the ansatz and then measurements are taken in the Pauli-Z basis
def rand_meas_circuit(pauli_op):
    rand_meas = QuantumCircuit(ansatz.num_qubits)
    for idx, op in enumerate(pauli_op):
        if op == 'X':
            rand_meas.h(idx)
        elif op == 'Y':
            rand_meas.h(idx)
            rand_meas.p(-np.pi/2, idx)
        elif op == 'Z':
            rand_meas.id(idx)
    return rand_meas



def objective_function(operators, params):
    
    
    # Assign parameters to the ansatz and simulate it
    # Generate circuits to measure random Paulis, one circuit for each Pauli
    
    # Putting repeated operators in derandomized_hamiltonian together and executing one single circuit 
    # as many times as the operator repetitions. 
    
    # convert the inner lists to tuples and count duplicates
    pauli_op_dict = Counter(tuple(x) for x in operators)
    
    
    shadow = []
    for pauli_op in pauli_op_dict:
        qc = ansatz.bind_parameters(params)
        qc.compose(rand_meas_circuit(pauli_op))
        qc.measure_all()
        result = execute(qc, backend, shots=pauli_op_dict[pauli_op]).result()
        counts = result.get_counts()
        
        
        # store the shadow in the form [[(Z,1),(Z,-1)...], [(Y,-1),(X,-1),...]] where inner list = snapshot
        # Because measurement output in Qiskit gives us states and not eigenvalues, we need to convert 0->1 and 1->-1
        
        for count in counts:
            for _ in range(counts[count]): # number of repeated measurement values
                output_str = list(count)
                output = [int(i) for i in output_str]
                eigenvals = [x+1 if x == 0 else x-2 for x in output]
                snapshot = [(op, eigenval) for op, eigenval in zip(pauli_op, eigenvals)]
                shadow.append(snapshot)
    
    
    # Now, we want to get the expectation values for the Hamiltonian from the shadow using the function
    # estimate_exp(full_measurement, one_observable)
    # where full_measurement = shadow and one_observable is any term in the Hamiltonian with I observable excluded.
    # cost = the total expectation value of the Hamiltonian
    # NOTE: We need to check for match_count value since it may happen that it is equal to 0. 
    # Such experiments need to be excluded. 
    # The problem is due to the derandomization algorithm not always producing Paulis that 'hit' the Hamiltonian terms.
    # NOTE: For term = [] we have that sum_product = match_count = len(shadow)
    
    cost = 0.0

    for term, weight in zip(hamiltonian_terms_XYZE, weights):
        sum_product, match_count = estimate_exp(shadow, term)
        if match_count != 0:
            exp_val = sum_product / match_count
            cost = cost + (weight * exp_val)     
        
    cost_history.append(cost)

        
    return cost


In [6]:
start_time = time.time()

measurement_range = [200]

for num_operators in measurement_range:   
    
    print('NUMBER OF OPERATORS:', num_operators)

    derandomized_hamiltonian = modified_derandomized_classical_shadow(hamiltonian_terms_XYZ, 
                                                                    num_operators, system_size, weight=abs_weights)

    cost_function = partial(objective_function, derandomized_hamiltonian)

    tuples = (tuple(pauli) for pauli in derandomized_hamiltonian)
    counts = Counter(tuples)
    print('DERANDOMISED OPERATORS:', counts)
    

    optimizer = SPSA(maxiter=1000)   
    expectation_values = []
    num_experiments = 10


    for iteration in range(num_experiments):
        cost_history = []
        params = np.random.rand(ansatz.num_parameters)
        result = optimizer.minimize(fun=cost_function, x0=params)
        expectation_values.append(min(cost_history)) 
        print("EXPERIMENT {}: GROUND STATE ENERGY FOUND = {}".format(iteration, min(cost_history)))



    rmse_derandomised_cs = np.sqrt(np.sum([(EXPECTED_EIGENVALUE - expectation_values[i])**2 
                                           for i in range(num_experiments)])/num_experiments)


    print('AVERAGE RMSE ERROR:', rmse_derandomised_cs)


elapsed_time = time.time() - start_time
print("Execution time = ", time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))




NUMBER OF OPERATORS: 200
DERANDOMISED OPERATORS: Counter({('X', 'X', 'X', 'X', 'X', 'X', 'X', 'X'): 199, ('Z', 'Z', 'Z', 'X', 'Z', 'Z', 'Z', 'X'): 1})
EXPERIMENT 0: GROUND STATE ENERGY FOUND = -1.3924099603043891
EXPERIMENT 1: GROUND STATE ENERGY FOUND = -1.3857978328664102
EXPERIMENT 2: GROUND STATE ENERGY FOUND = -1.3984105366614386
EXPERIMENT 3: GROUND STATE ENERGY FOUND = -1.3902745960259746
EXPERIMENT 4: GROUND STATE ENERGY FOUND = -1.3930411042412225
EXPERIMENT 5: GROUND STATE ENERGY FOUND = -1.3920117383625565
EXPERIMENT 6: GROUND STATE ENERGY FOUND = -1.4041852596161912
EXPERIMENT 7: GROUND STATE ENERGY FOUND = -1.3881045726180115
EXPERIMENT 8: GROUND STATE ENERGY FOUND = -1.3968140982582278
EXPERIMENT 9: GROUND STATE ENERGY FOUND = -1.370397106553329
AVERAGE RMSE ERROR: 0.4689331611731092
Execution time =  00:22:09


In [8]:
start_time = time.time()

measurement_range = [1000]

for num_operators in measurement_range:   
    
    print('NUMBER OF OPERATORS:', num_operators)

    derandomized_hamiltonian = modified_derandomized_classical_shadow(hamiltonian_terms_XYZ, 
                                                                    num_operators, system_size, weight=abs_weights)

    cost_function = partial(objective_function, derandomized_hamiltonian)

    tuples = (tuple(pauli) for pauli in derandomized_hamiltonian)
    counts = Counter(tuples)
    print('DERANDOMISED OPERATORS:', counts)
    

    optimizer = SPSA(maxiter=2000)   
    expectation_values = []
    num_experiments = 1


    for iteration in range(num_experiments):
        cost_history = []
        params = np.random.rand(ansatz.num_parameters)
        result = optimizer.minimize(fun=cost_function, x0=params)
        expectation_values.append(min(cost_history)) 
        print("EXPERIMENT {}: GROUND STATE ENERGY FOUND = {}".format(iteration, min(cost_history)))



    rmse_derandomised_cs = np.sqrt(np.sum([(EXPECTED_EIGENVALUE - expectation_values[i])**2 
                                           for i in range(num_experiments)])/num_experiments)


    print('AVERAGE RMSE ERROR:', rmse_derandomised_cs)


elapsed_time = time.time() - start_time
print("Execution time = ", time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))




NUMBER OF OPERATORS: 1000
DERANDOMISED OPERATORS: Counter({('X', 'X', 'X', 'X', 'X', 'X', 'X', 'X'): 995, ('Z', 'Z', 'Z', 'X', 'Z', 'Z', 'Z', 'X'): 3, ('Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z'): 2})
EXPERIMENT 0: GROUND STATE ENERGY FOUND = -1.8205641759103157
AVERAGE RMSE ERROR: 0.03943582408968438
Execution time =  00:08:06
